dictSKB

In [1]:
from math import log
import tqdm
import json
import numpy as np
import matplotlib.pyplot as plt

In [81]:
core_sememe_dict = np.load("./core_sememe_dict.npy", allow_pickle=True).tolist()
dict_sememes = np.load("./dict_sememes.npy", allow_pickle=True).tolist()

In [83]:
len(core_sememe_dict)

68860

In [82]:
dict_sememes = {w:i for i, w in enumerate(dict_sememes)}

In [85]:
dictSKB_68860 = {}
for word, sence in core_sememe_dict.items():
    for (_, sememes) in sence:
        if len(sememes) == 0:
            continue
        if word in dictSKB_68860.keys():
            continue
        dictSKB_68860[word] = sememes

In [90]:
dictSKB_68860

{'000': {'call', 'use'},
 '1': {'message'},
 '1066': {'date'},
 '12': {'allow'},
 '1471': {'call'},
 '15': {'allow'},
 '18': {'accept', 'letter'},
 '18-wheeler': {'carry', 'distance', 'use'},
 '2': {'email', 'message', 'use', 'way'},
 '2.0': {'improve', 'use'},
 '20/20 vision': {'ability', 'see', 'wear'},
 '20th Century Fox': {'company'},
 '2:1': {'level', 'university'},
 '.22': {'animal', 'fire', 'gun'},
 '2:2': {'level', 'university'},
 '24': {'series', 'show', 'unit'},
 '24-7': {'happen'},
 '24 hour clock': {'call', 'clock', 'hour'},
 '3-D': {'length'},
 '3-D printer': {'design', 'layer', 'piece', 'substance'},
 '3-D printing': {'build', 'design', 'make', 'substance'},
 '3G': {'fast', 'phone', 'short', 'video'},
 '4': {'email', 'message', 'use', 'way'},
 '401K': {'handle', 'way'},
 '404': {'search', 'stupid', 'word'},
 '411': {'find'},
 '419 scam': {'exist', 'make'},
 '.45': {'gun'},
 '4-D film': {'film'},
 '4-F': {'allow', 'member'},
 '4G': {'phone', 'provide'},
 '4-H': {'citizen'}

In [86]:
len(dictSKB_68860)

68860

In [91]:
graph = open("dictSKB_68860.txt", 'w')

for word, sememes in dictSKB_68860.items():
    if len(sememes) == 0:
        continue
    for sememe in sememes:
        graph.writelines(word+"\t"+str(dict_sememes[sememe])+"\t")
        graph.write('\n') 
        graph.writelines(str(dict_sememes[sememe])+"\t"+word+"\t")
        graph.write('\n')

graph.close()

hownet

In [25]:
import numpy as np
import OpenHowNet

In [26]:
#OpenHowNet.download()

In [27]:
hownet_dict = OpenHowNet.HowNetDict()

In [28]:
hownet_sememes = np.load("./hownet_sememes.npy", allow_pickle=True).tolist()

In [29]:
hownet_sememes = {w:i for i, w in enumerate(hownet_sememes)}

In [30]:
hownet_words = hownet_dict.get_en_words()

In [31]:
len(hownet_words)

118261

In [32]:
graph = open("hownet_graph.txt", 'w')
for word in hownet_words:
    
    sememes = hownet_dict.get_sememes_by_word(word,structured=False,lang="en",merge=True)
    if len(sememes) == 0:
        print(1)
        continue
    for sememe in sememes:
        if sememe not in hownet_sememes.keys():
            continue
        graph.writelines(word+"\t"+str(hownet_sememes[sememe])+"\t"+str(1))
        graph.write('\n')
        graph.writelines(str(hownet_sememes[sememe])+"\t"+word+"\t"+str(1))
        graph.write('\n')
    
graph.close()

own

In [16]:
sememe_network_dict = np.load("./sememe_network_dict_en_wordnet_all_tagme_2000.npy", allow_pickle=True).tolist()
sememe_network_cdv = np.load("./sememe_network_cdv_en_wordnet_all_tagme_2000.npy", allow_pickle=True).tolist()

In [12]:
import re
def titleProcessing(title):
    #title = re.findall(r"title=\"(.*)\">", title)
    #title = title[0].replace(" ", "_")
    return re.findall(r"title=\"(.*)\">", title)

index_content_dic = {}
title_list = []
count = -1
for line in open('../data/wiki.txt'):
    if re.match('<doc id.*>', line):
        title_list.append(titleProcessing(line))
        count+=1
        index_content_dic[count] = []
        continue
    index_content_dic_value = index_content_dic[count]
    index_content_dic_value.append(line)
    index_content_dic[count] = index_content_dic_value

In [13]:
len(index_content_dic)

6403704

In [7]:
# this part for pmi graph:
sentences_all_text = []
for items in index_content_dic.values():
    for item in items[:2]:
        sentences_all_text.append(item.strip())
index_content_dic = {}
title_list = []

In [8]:
# build vocab
word_freq = {}
word_set = set()
for doc_words in tqdm.tqdm(sentences_all_text):
    words = doc_words.split()
    for word in words:
        word_set.add(word)
        if word in word_freq:
            word_freq[word] += 1
        else:
            word_freq[word] = 1

vocab = list(word_set)
vocab_size = len(vocab)
word_id_map = {}
for i in range(vocab_size):
    word_id_map[vocab[i]] = i

0it [00:00, ?it/s]


In [10]:
sentences_all_text

[]

In [ ]:
# word co-occurence with context windows
window_size = 20
windows = []
print("part1")
for doc_words in tqdm.tqdm(sentences_all_text):
    words = doc_words.split()
    length = len(words)
    if length <= window_size:
        windows.append(words)
    else:
        # print(length, length - window_size + 1)
        for j in range(length - window_size + 1):
            window = words[j: j + window_size]
            windows.append(window)
            # print(window)

part1


  5%|█▎                         | 2676820/54814140 [04:13<129:00:11, 112.27it/s]

In [ ]:
word_window_freq = {}
for window in windows:
    appeared = set()
    for i in range(len(window)):
        if window[i] in appeared:
            continue
        if window[i] in word_window_freq:
            word_window_freq[window[i]] += 1
        else:
            word_window_freq[window[i]] = 1
        appeared.add(window[i])

In [ ]:
print("part2")
word_pair_count = {}
for window in windows:
    for i in range(1, len(window)):
        for j in range(0, i):
            word_i = window[i]
            word_i_id = word_id_map[word_i]
            word_j = window[j]
            word_j_id = word_id_map[word_j]
            if word_i_id == word_j_id:
                continue
            word_pair_str = str(word_i_id) + ',' + str(word_j_id)
            if word_pair_str in word_pair_count:
                word_pair_count[word_pair_str] += 1
            else:
                word_pair_count[word_pair_str] = 1
            # two orders
            word_pair_str = str(word_j_id) + ',' + str(word_i_id)
            if word_pair_str in word_pair_count:
                word_pair_count[word_pair_str] += 1
            else:
                word_pair_count[word_pair_str] = 1
num_window = len(windows)

In [38]:

    



print("part3")

pmi_graph = open("word.graph", 'w')

for key in word_pair_count:
    temp = key.split(',')
    i = int(temp[0])
    j = int(temp[1])
    count = word_pair_count[key]
    word_freq_i = word_window_freq[vocab[i]]
    word_freq_j = word_window_freq[vocab[j]]
    pmi = log((1.0 * count / num_window) /
              (1.0 * word_freq_i * word_freq_j/(num_window * num_window)))
    if pmi <= 0:
        continue
    pmi_graph.writelines(vocab[i]+" "+vocab[j]+" "+str(pmi))
    pmi_graph.write('\n')
pmi_graph.close()


KeyboardInterrupt: 

In [ ]:

pmi_graph = open("pmi_graph", 'w')

for key in word_pair_count:
    temp = key.split(',')
    i = int(temp[0])
    j = int(temp[1])
    count = word_pair_count[key]
    word_freq_i = word_window_freq[vocab[i]]
    word_freq_j = word_window_freq[vocab[j]]
    pmi = log((1.0 * count / num_window) /
              (1.0 * word_freq_i * word_freq_j/(num_window * num_window)))
    if pmi <= 0:
        continue
    pmi_graph.writelines(vocab[i]+"\t"+vocab[j]+"\t"+str(pmi))
    pmi_graph.write('\n')
    pmi_graph.writelines(vocab[j]+"\t"+vocab[i]+"\t"+str(pmi))
    pmi_graph.write('\n')
pmi_graph.close()

In [17]:
sememe_network_cdv = {w:i for i, w in enumerate(sememe_network_cdv)}

In [18]:
graph = open("own_graph.txt", 'w')

for word, sence in sememe_network_dict.items():
    word = word.split(".")[:-2]
    for sememe in sence:
        graph.writelines("".join(word)+"\t"+sememe+"\t"+str(1))
        graph.write('\n')
        #graph.writelines(sememe+"\t"+"".join(word)+"\t"+str(1))
        #graph.write('\n')

graph.close()

wiki

In [2]:
import numpy as np

In [3]:
sememe_network_wiki_all_pmi_dict = np.load("./sememe_network_wiki_all_pmi_dict.npy", allow_pickle=True).tolist()

In [4]:
"disambiguation"

'disambiguation'

In [5]:
len(sememe_network_wiki_all_pmi_dict)

800458

In [6]:
word_sememe_dict_wiki = {}
for title, content in sememe_network_wiki_all_pmi_dict.items():
    word, pos = title.split(">>>")
    if " (" not in word and word not in word_sememe_dict_wiki.keys():
        word_sememe_dict_wiki[word] = content

    if "disambiguation" in word and word not in word_sememe_dict_wiki.keys():
        word_sememe_dict_wiki[word.split(" (")[0]] = content

    if word.split(" (")[0] not in word_sememe_dict_wiki.keys():
        word_sememe_dict_wiki[word.split(" (")[0]] = content

In [7]:
len(word_sememe_dict_wiki)

697754

In [8]:
graph = open("wiki_697754_573.txt", 'w')

for word, sememes in word_sememe_dict_wiki.items():
    for sememe in sememes:
        sememe, pmi = sememe.split(":")
        graph.writelines(word+"\t"+sememe+"\t"+str(pmi))
        graph.write('\n')
        graph.writelines(sememe+"\t"+word+"\t"+str(pmi))
        graph.write('\n')


graph.close()

RH

In [76]:
sememe_network_wordnet_all_pmi_dict = np.load("./wordnet_sememe_163340_218.npy", allow_pickle=True).tolist()

In [77]:
len(sememe_network_wordnet_all_pmi_dict)

137601

In [78]:
word_sememe_dict_wordnet = {}
for title, content in sememe_network_wordnet_all_pmi_dict.items():
    word = title.split(".")[:-2]
    if "".join(word) not in word_sememe_dict_wordnet.keys():
        word_sememe_dict_wordnet["".join(word)] = content


In [79]:
len(word_sememe_dict_wordnet)

101994

In [70]:
graph = open("wordnet_163340_218.txt", 'w')

for word, sememes in word_sememe_dict_wordnet.items():
    for sememe in sememes:
        sememe, pmi = sememe.split(":")
        graph.writelines(word+"\t"+sememe+"\t"+str(pmi))
        graph.write('\n')
        graph.writelines(sememe+"\t"+word+"\t"+str(pmi))
        graph.write('\n')


graph.close()